In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

## <font color = 'green'> Solution 1 approach </font>

### Since average seats operated by Cityflo is increasing on further days whereas Avg rides and occupancy is decreasing, we can either reduce a bus or a route depending on what caters to maximum customers while minimizing the operational cost most

In [244]:
df_cust = pd.read_excel('customers.xlsx')

import datetime as dt

df_cust['Week'] = df_cust['Ride Date'].dt.week

df_cust.head()

,Customer Name,Ride Date,Week
0,Natasha,2020-01-09,2
1,Karan,2020-01-03,1
2,Arakkal,2020-01-10,2
3,Arakkal,2020-01-02,1
4,Padegaonkar,2020-02-10,7


## <font color = 'green'> Solution 2 </font>

In [243]:
a = df_cust.groupby(['Customer Name','Week']).count().unstack().reset_index()
a.columns = a.columns.get_level_values(1)
a = a.rename(columns={'':'Name',a.columns[1]:'Week 1',a.columns[2]:'Week 2',a.columns[3]:'Week 3',a.columns[4]:'Week 4',a.columns[5]:'Week 5',a.columns[6]:'Week 6',a.columns[7]:'Week 7',a.columns[8]:'Week 8',a.columns[9]:'Week 9',a.columns[10]:'Week 10',a.columns[11]:'Week 11'})
a = a.fillna(0)

for j in range(0,21):
    if (a.iloc[j,1] != 0):
        a.iloc[j,1] = 'New Customer'
    else:
        a.iloc[j,1] = 0

for i in range(2,12):
    num = 0
    for j in range(0,21):
        if (a.iloc[j,i] > 0 and a.iloc[j,i-1] == 'New Customer'):
            a.iloc[j,i] = 1
        elif (a.iloc[j,i] > 0 and a.iloc[j,i-1] <= -1):
            a.iloc[j,i] = 1
        elif (a.iloc[j,i] > 0 and a.iloc[j,i-1] > 0):
            a.iloc[j,i] = a.iloc[j,i-1] + 1
        elif (a.iloc[j,i] == 0 and a.iloc[j,i-1] == 'New Customer'):
            a.iloc[j,i] = -1
        elif (a.iloc[j,i] == 0 and a.iloc[j,i-1] > 0):
            a.iloc[j,i] = -1
        elif (a.iloc[j,i] > 0 and a.iloc[j,i-1] == 0):
            for k in range(1,i):
                if (a.iloc[j,i-k-1] == 0):
                    num+=1
            if (num == i-2):
                a.iloc[j,i] = 'New Customer'
        elif (a.iloc[j,i] == 0 and a.iloc[j,i-1] <= -1):
            a.iloc[j,i] = a.iloc[j,i-1] -1
        else : 
            a.iloc[j,i] = 0
a = a.astype(int, errors='ignore')
a = a.replace(0, "-")
a

Week,Name,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11
0,Agaskar,-,New Customer,1,2,3,4,5,6,7,8,9
1,Agrawal,-,-,New Customer,1,2,-1,-2,1,-1,1,-1
2,Arakkal,New Customer,1,2,-1,-2,1,2,-1,1,2,-1
3,Bangera,-,New Customer,-1,-2,-3,-4,-5,-6,1,-1,-2
4,Chaturvedi,New Customer,1,2,3,4,5,6,7,8,-1,-2
5,Haldipur,-,New Customer,-1,-2,-3,-4,1,-1,1,-1,-2
6,Jolly,-,-,7,8,9,-1,1,2,3,-1,-2
7,Karan,New Customer,-1,-2,-3,-4,-5,-6,1,2,-1,-2
8,Kaushal,New Customer,1,2,3,4,5,6,7,8,-1,-2
9,Kelkar,-,-,1,-1,-2,-3,-4,-5,-6,-7,-8


In [155]:
df_pick = pd.read_excel('DA_test.xlsx',sheet_name=1)

In [178]:
locations = df_pick[['home_location_lat', 'home_location_long']]
locationlist = locations.values.tolist()
map_ = folium.Map(location=[19, 73], zoom_start=10)
for point in range(0, 1000):
    folium.Marker(locationlist[point], popup=df_pick['customer_id'][point]).add_to(map_)
map_

### Lets visualize the clusters of pickup together and find the ditance between extreme customers

### Lets create a funnction to calculate distance between two customers

In [148]:
def dist(cust1_lat,cust1_lon,cust2_lat,cust2_lon):
    R = 6373.0  #radius of the Earth

    lat1 = math.radians(cust1_lat) #coordinates
    lon1 = math.radians(cust1_lon)
    lat2 = math.radians(cust2_lat)
    lon2 = math.radians(cust2_lon)

    dlon = lon2 - lon1 #change in coordinates
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2 
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return round(distance,2)

### Between customer 145851 and 122506

In [139]:
dist(df_pick.home_location_lat[df_pick['customer_id']==145851],df_pick.home_location_long[df_pick['customer_id']==145851],df_pick.home_location_lat[df_pick['customer_id']==122506],df_pick.home_location_long[df_pick['customer_id']==122506])

14.446304026827047

### Lets try to locate 7 almost exclusive clusters in between these extreme customers, we want the clusters to be equidistant as long as it covers an optimal number of customers, lets also try to control the walking distance while choosing these locations

### Lets start with one of the extreme, distance between 106918 and 145851

In [145]:
dist(df_pick.home_location_lat[df_pick['customer_id']==106918],df_pick.home_location_long[df_pick['customer_id']==106918],df_pick.home_location_lat[df_pick['customer_id']==145851],df_pick.home_location_long[df_pick['customer_id']==145851])

2.7783867962449778

### Lets try and locate a stop in between 106918 and 145851 so as to minimize walking distance from both of them

In [146]:
dist(df_pick.home_location_lat[df_pick['customer_id']==145851],df_pick.home_location_long[df_pick['customer_id']==145851],df_pick.home_location_lat[df_pick['customer_id']==112048],df_pick.home_location_long[df_pick['customer_id']==112048])

1.60009249943924

In [147]:
dist(df_pick.home_location_lat[df_pick['customer_id']==106918],df_pick.home_location_long[df_pick['customer_id']==106918],df_pick.home_location_lat[df_pick['customer_id']==112048],df_pick.home_location_long[df_pick['customer_id']==112048])

1.6771011239928817

### A stop near the pick up of customer 106918 should be the ideal first stop, ofcourse this is one of the local solutions to the problem but goin further lets try to create an algorithm that provides a global solution to the problem

## <font color = 'green'> Solution 3 approach </font>

The problem could have been solved using traditional clustering algorithms such as density based on mean based clustering but the cost function that these model optimizes are more of a mathematical function and does not incorporate business understanding. The resulting clusters reflect this as in this particular case, the cluster forms skewed distribution with a cluster near Thane location consisting of majority of customers. We can tweak the number of clusters to overcome this problem or come up with a different cost function altogether to optimize.

Below I have tried one such algorithm - 

Find and store distance between all other customers pick up location for a given customer

Try different distance thresholds such as 0.5, 1 and 1.5 kms

Store the ids of customers that this customer location contains as related customer ids and later count the number of customers covered by this locatn

arrange customer ids with respect to number of customers covered

Iterate through the customer ids by selecting the first customer id as a possible stop and looking for other customer ids that does not belong to the related customer ids of the customer id

Keep adding customer ids in similar format untill the iteration is complete

Check for number of stops, number of customers, customers per stop and customers covered out of total customers and store this information along with customer ids that will be used as a possible stop

Repeat the same process by starting with say first 10 customer ids by maximum number of customers covered and record the above summary statistics for all of them separately

Optimize for number of stops, customers covered out of total customers and number of customers per stop incorporating Business understanding and cost-benefit analysis.

For the customers not covered in the stops formed, it is advisable to try including them in one of the stops if walking distance is manageable or create a stop for them, take a burn in initial few days and work on getting more customers around that area for operations in that route to turn profitable

This problem is all about finding a global solution amidst tuning of parameters. Tuning these parameters can be tricky but with a decent business understanding, a brute force optimization can be performed and after further brainstorming a global solution can be reached

In [174]:
import folium
from folium.plugins import MarkerCluster
COORDINATES = (19, 73)
gdata = df_pick
map_nyctaxi = folium.Map(location=COORDINATES, zoom_start=10)
marker_cluster = MarkerCluster().add_to(map_nyctaxi)
for each in gdata[0:1000].iterrows():
    folium.Marker(
        location = [each[1]['home_location_lat'],each[1]['home_location_long']], popup='picked here',icon=folium.Icon(color='red')).add_to(marker_cluster)
map_nyctaxi

In [5]:
df_pick_drop = pd.read_excel('DA_test.xlsx',sheet_name=2)

In [173]:
import folium
from folium.plugins import MarkerCluster
NY_COORDINATES = (19, 73)
gdata = df_pick_drop
# MAX_RECORDS = 1048576
map_nyctaxi = folium.Map(location=NY_COORDINATES, zoom_start=10)
marker_cluster = MarkerCluster().add_to(map_nyctaxi)
for each in gdata[0:100].iterrows():
    folium.Marker(
        location = [each[1]['home_location_lat'],each[1]['home_location_long']], popup='picked here',icon=folium.Icon(color='red')).add_to(marker_cluster)
    folium.Marker(
        location = [each[1]['office_location_lat'],each[1]['office_location_long']], popup='dropped here',icon=folium.Icon(color='green')).add_to(marker_cluster)
map_nyctaxi

In [35]:
pickup_coordinates = []
for i in range(100):
    pickup_coordinates.append((df_pick_drop.home_location_lat[i],df_pick_drop.home_location_long[i]))
    i+= 1
dropoff_coordinates = []
for i in range(100):
    dropoff_coordinates.append((df_pick_drop.office_location_lat[i],df_pick_drop.office_location_long[i]))
    i+= 1

In [36]:
all_coordinates = list(zip(pickup_coordinates,dropoff_coordinates))

In [37]:
def plot_trip(coordinates):
    for coordinate in coordinates:
        p1 = [coordinate[0][0],coordinate[0][1]]
        p2 = [coordinate[1][0],coordinate[1][1]]
        folium.PolyLine(locations=[p1,p2],smooth_factor=0.1).add_to(map_)

In [168]:
map_ = folium.Map(location=[19, 73], max_zoom=12)
plot_trip(all_coordinates)
map_

## <font color = 'green'> Solution 4 approach </font>

Iterate latitude values from minimum to maximum

Iterate longitude values from minimum to maximum 

check if (max(home latitude) < min(office latitude)) or (max(home longitude) < min(office longitude)) or (min(home latitude) < max(office latitude)) or (min(home longitude) < max(office longitude))

check if ((home latitude and longitude and office latitude and longitude is between latitude and longitude)) and (pick and drop stop latitude and longitude is between latitude and longitude)

Calculate following metrics - 

Customers covered, number of pickup stops, number of drop off stops, customer per stop

Keep appending these metrics in a dictionary or a dataframe along with lat, long of the route and create a                 unique route id for all routes

arrange route ids with respect to number of customers covered

Iterate through the route ids by selecting the first route and then looking for other routes

Keep adding routes in similar format untill all customers are covered

Check for number of routes, Average number of pickup and dropoff stops per route and create a unique network id to store this information

Repeat the same process by starting with say first 10 route ids by maximum number of customers covered and record the above summary statistics for all of them separately

Optimize for number of routes, Average number of pickup and dropoff stops per route incorporating Business understanding and cost-benefit analysis.

This problem is all about finding a global solution amidst tuning of parameters. Tuning these parameters can be tricky but with a decent business understanding, a brute force optimization can be performed and after further brainstorming a global solution can be reached

## <font color = 'green'> Solution 5 approach </font>

### For the new Revenue to be more than current revenue, the price elasticity of demand should be more than unit elastic. For example, if the fare is increased by 10%, the customers should decrease by less than 10/11% for revenue to be more than the current revenue. Since, this depicts a linear relationship, this could be modelled using linear programming where Y depicts the increase in fare and x depicts decrease in customers, the relationship is expected to be negative linear one